In [1]:
import pandas as pd
import numpy as np
import os, sys
# import matplotlib.pyplot as plt
# import seaborn as sns
import json
sys.path.append('../')
from models.utils import rocksdb_knobs_make_dict

In [2]:
PATH_internal = "../data/internal"
PATH_external = "../data/external"
PATH_knobs = "../data/rocksdb_conf"
wk_len = 16

In [3]:
knobs = rocksdb_knobs_make_dict(PATH_knobs)
knobs = pd.DataFrame(data=knobs['data'].astype(np.float32), columns=knobs['columnlabels'])
columns = knobs.columns
knobs.head(4)

,max_background_compactions,max_background_flushes,write_buffer_size,max_write_buffer_number,min_write_buffer_number_to_merge,compaction_pri,compaction_style,level0_file_num_compaction_trigger,level0_slowdown_writes_trigger,level0_stop_writes_trigger,...,open_files,block_size,cache_index_and_filter_blocks,max_bytes_for_level_base,max_bytes_for_level_multiplier,target_file_size_base,target_file_size_multiplier,num_levels,memtable_bloom_size_ratio,compression_ratio
0,6.0,16.0,1304576.0,3.0,2.0,1.0,0.0,3.0,24.0,44.0,...,10000.0,9216.0,0.0,5242880.0,11.0,1800192.0,1.0,8.0,0.10,0.67
1,2.0,14.0,911360.0,6.0,2.0,0.0,0.0,7.0,18.0,55.0,...,100000.0,15360.0,1.0,3145728.0,10.0,1729536.0,1.0,5.0,0.15,0.50
2,8.0,6.0,1474560.0,6.0,1.0,2.0,0.0,5.0,18.0,48.0,...,1000000.0,6144.0,0.0,4194304.0,9.0,1429504.0,1.0,6.0,0.05,0.36
3,12.0,8.0,550912.0,7.0,1.0,3.0,0.0,4.0,18.0,35.0,...,10000.0,3072.0,1.0,2097152.0,11.0,900096.0,1.0,7.0,0.05,0.63


In [44]:
wk_len = 16
internal_dict = {}

pruned_im = pd.read_csv(os.path.join(PATH_internal, 'internal_ensemble_pruned_tmp.csv'), index_col=0)
for wk in range(wk_len):
    im = pd.read_csv(os.path.join(PATH_internal, f'internal_results_{wk}.csv'), index_col=0)
    internal_dict[wk] = im[pruned_im.columns]
#     break
internal_dict[0].head(4)

,rocksdb.block.cache.miss COUNT,rocksdb.block.cache.hit COUNT,rocksdb.block.cache.add COUNT,rocksdb.block.cache.index.miss COUNT,rocksdb.block.cache.index.hit COUNT,rocksdb.block.cache.index.add COUNT,rocksdb.block.cache.index.bytes.insert COUNT,rocksdb.block.cache.data.miss COUNT,rocksdb.block.cache.data.hit COUNT,rocksdb.block.cache.data.add COUNT,...,rocksdb.bytes.decompressed_P99,rocksdb.bytes.decompressed_P100,rocksdb.bytes.decompressed_COUNT,rocksdb.bytes.decompressed_SUM,rocksdb.db.flush.micros_P50,rocksdb.db.flush.micros_P95,rocksdb.db.flush.micros_P99,rocksdb.db.flush.micros_P100,rocksdb.db.flush.micros_COUNT,rocksdb.db.flush.micros_SUM
index,,,,,,,,,,,,,,,,,,,,,
/home/jieun/data_generation_rocksdb/rocksdb_conf/config1.cnf,1296845,1172926,1213729,0,0,0,0,1296845,1172926,1213729,...,8383.000000,8383.0,1297251,10856979511,21652.173913,30615.000000,30615.0,30615.0,44,1015663
/home/jieun/data_generation_rocksdb/rocksdb_conf/config2.cnf,1304820,4192080,1256074,1582,2701880,1582,5078096,1303238,1490200,1254492,...,14659.000000,14659.0,1304820,19074632333,18135.593220,21979.661017,26561.0,26561.0,63,1074316
/home/jieun/data_generation_rocksdb/rocksdb_conf/config3.cnf,1407632,2195141,1315590,0,0,0,0,1407632,2195141,1315590,...,6578.256132,12483.0,1407897,8847109704,29660.714286,45820.000000,45820.0,45820.0,78,2692570
/home/jieun/data_generation_rocksdb/rocksdb_conf/config4.cnf,1497592,7007880,1252738,974,4003817,974,6993856,1496618,3004063,1251764,...,4385.648850,9451.0,1497592,4718848784,24646.616541,32387.969925,34773.0,34773.0,208,5221234


In [5]:
external_dict = {}
for wk in range(wk_len):
    ex = pd.read_csv(os.path.join(PATH_external, f'external_results_{wk}.csv'), index_col=0)
    external_dict[wk] = ex
    break
external_dict[0].head(4)

,TIME,RATE,WAF,SA
index,,,,
/home/jieun/data_generation_rocksdb/rocksdb_conf/config1.cnf,21.8,4.77,8.7,72.17
/home/jieun/data_generation_rocksdb/rocksdb_conf/config2.cnf,25.9,4.02,8.9,55.83
/home/jieun/data_generation_rocksdb/rocksdb_conf/config3.cnf,52.0,2.00,7.5,34.21
/home/jieun/data_generation_rocksdb/rocksdb_conf/config4.cnf,53.6,1.94,10.5,56.94


In [6]:
from scipy.spatial import distance
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [7]:
def euclidean_distance(a, b):
    res = a - b
    res = res ** 2
    res = np.sqrt(res)
#     return res
    return np.average(res)

In [8]:
scaler = MinMaxScaler().fit(pd.concat(internal_dict))

In [152]:
def get_euclidean_distance(wk_num, size):
    wk = []
    rand_idx = np.random.choice(np.arange(20000), size) # 
    trg_rand_idx = np.random.choice(np.arange(20000), size)
    
    for im_d in internal_dict:
        if im_d == wk_num:
            wk.append(scaler.transform(internal_dict[im_d].iloc[trg_rand_idx, :]))
        else:
            wk.append(scaler.transform(internal_dict[im_d].iloc[rand_idx, :]))
#         wk.append(scaler.transform(internal_dict[im_d].iloc[:size, :]))
        
    big = 100
    for i in range(16):
        ed = euclidean_distance(wk[wk_num], wk[i])
        if ed<big and wk_num != i: 
            big=ed
            idx = i
        print(f'{i:4}th   {ed}')
    print('best similar workload is ',idx, 'th')

In [38]:
internal_dict.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])

In [56]:
def get_euclidean_distance_rand(target, size):
    target_wk = pd.read_csv(f'../data/target_workload/{target}/internal_results_11.csv', index_col=0)
    target_wk = target_wk[pruned_im.columns]

    wk = []
    rand_idx = np.random.choice(np.arange(20000), size)
    trg_rand_idx = np.random.choice(np.arange(20), size)

    for im_d in internal_dict:
        wk.append(scaler.transform(internal_dict[im_d].iloc[rand_idx, :]))

    target_wk = scaler.transform(target_wk.iloc[trg_rand_idx, :])

    big = 100
    for i in range(16):
        ed = euclidean_distance(target_wk, wk[i])
        if ed<big and wk_num != i: 
            big=ed
            idx = i
        print(f'{i:4}th   {ed}')
    print('best similar workload is ',idx, 'th')

In [278]:
get_euclidean_distance_rand(16, 20)

   0th   0.05878163615922685
   1th   0.03240533829089025
   2th   0.0347304600429611
   3th   0.07985093322700512
   4th   0.08735179794956742
   5th   0.05215369291778668
   6th   0.05135089267078403
   7th   0.10353100711858015
   8th   0.11963497030630516
   9th   0.08844596068661495
  10th   0.0836030414409341
  11th   0.13651085972812596
  12th   0.19171321603607572
  13th   0.11722804738441488
  14th   0.10062668333471941
  15th   0.15233661638209006
best similar workload is  1 th


In [299]:
get_euclidean_distance_rand(17, 20)

   0th   0.07510108428835212
   1th   0.05162800078094276
   2th   0.051544505134127734
   3th   0.0971829110158212
   4th   0.0875670648870535
   5th   0.050887776935470155
   6th   0.05079740481633043
   7th   0.1053050015541088
   8th   0.10449186124111635
   9th   0.06766239664457824
  10th   0.06740876958837319
  11th   0.12311885660315901
  12th   0.1818398313367089
  13th   0.10349802206092082
  14th   0.08582679201030484
  15th   0.13935271758857048
best similar workload is  6 th


In [318]:
get_euclidean_distance_rand(18, 20)

   0th   0.09169819947985554
   1th   0.07290686336167694
   2th   0.07761869874493245
   3th   0.12614907095721942
   4th   0.08843341148599534
   5th   0.05856930154188499
   6th   0.06864388882649868
   7th   0.1265840402584076
   8th   0.09661574314195612
   9th   0.06791113948411621
  10th   0.07239070009051159
  11th   0.13339182042101255
  12th   0.13440679158106855
  13th   0.06309600854891739
  14th   0.05688573550757765
  15th   0.11705916698322372
best similar workload is  14 th


In [319]:
get_euclidean_distance(5, 10)

   0th   0.0734461047169697
   1th   0.05548962720790027
   2th   0.0650508932311016
   3th   0.1123254944415243
   4th   0.06235384687684123
   5th   0.0
   6th   0.051627930152499335
   7th   0.1080243101516575
   8th   0.09409748692219508
   9th   0.06738019832972203
  10th   0.07750751332890499
  11th   0.13402095483971135
  12th   0.15685021636405502
  13th   0.08590687211863778
  14th   0.08764987367648089
  15th   0.14648626088034114
best similar workload is  6 th


In [216]:
get_euclidean_distance(0, 50)

   0th   0.0
   1th   0.03839342778518646
   2th   0.05537281187298851
   3th   0.10452375177966208
   4th   0.07899344781142488
   5th   0.07427691109109276
   6th   0.0893821860106895
   7th   0.1450092709293601
   8th   0.13335757893125424
   9th   0.10838439069299927
  10th   0.11600908967719706
  11th   0.17446573829490947
  12th   0.15898829435748751
  13th   0.1286883930895689
  14th   0.13458126735581769
  15th   0.19361380028498706
best similar workload is  1 th


In [163]:
get_euclidean_distance(0, 100)

   0th   0.0
   1th   0.03881406290314002
   2th   0.05604497650874244
   3th   0.10358141199610259
   4th   0.08167467602097779
   5th   0.07573355227728042
   6th   0.08998560722673835
   7th   0.14416242191391224
   8th   0.13570585130653687
   9th   0.11071626994084872
  10th   0.11783882714931684
  11th   0.17281327708928
  12th   0.16420907008508726
  13th   0.1322669128285695
  14th   0.13491799729614876
  15th   0.1925845191597016
best similar workload is  1 th


In [164]:
get_euclidean_distance(0, 1000)

   0th   0.0
   1th   0.038716974740942586
   2th   0.0559694928139756
   3th   0.10353402059139667
   4th   0.0819679428221237
   5th   0.07623963789366729
   6th   0.0905833508968693
   7th   0.14455975591828532
   8th   0.1362268469328294
   9th   0.11063566479864688
  10th   0.11790948826567332
  11th   0.17347230216448226
  12th   0.1644112923597811
  13th   0.13273552423230378
  14th   0.13599648673005438
  15th   0.19328209185173864
best similar workload is  1 th
